In [1]:
%matplotlib inline

In [2]:
# %load poi_id.py
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi". # You will need to use more features
##### 'email_address' fails featureFormat() as it is string not float, all other features imported for 
##### initial testing and investigations
features_list = ['poi', 'salary', 'bonus','deferral_payments','total_payments',  
                 'exercised_stock_options','restricted_stock','total_stock_value',
                 'expenses','deferred_income',
                 'long_term_incentive', 'other',
                 'restricted_stock_deferred', 'loan_advances', 'director_fees', 
                 'to_messages', 'shared_receipt_with_poi','from_messages',      
                 'from_this_person_to_poi', 'from_poi_to_this_person'             
                ] 

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

    
for namen in data_dict:
    count = 0
    for things, values in data_dict[namen].items():
        if values == 'NaN':
            count +=1
    if count >=18:
        print namen, count
        
### Task 2: Remove outliers - remove TOTAL, Lockhardt Eugene E (all NaNs), and The Agency in the Park    
outlier = data_dict.pop('TOTAL')
outlier = data_dict.pop('LOCKHART EUGENE E')
outlier = data_dict.pop('THE TRAVEL AGENCY IN THE PARK')

### Negative outliers found in deferred_income and restricted stock deferred
### Entries for 'BELFER ROBERT' & 'BHATANGAR SANJAY' corrected as confirmed by enron61702insiderpay.pdf

data_dict['BELFER ROBERT']['deferred_income']  = -102500
data_dict['BELFER ROBERT']['deferral_payments']= 'NaN'
data_dict['BELFER ROBERT']['expenses'] = 3285
data_dict['BELFER ROBERT']['director_fees'] = 102500 
data_dict['BELFER ROBERT']['total_payments'] = 3285
data_dict['BELFER ROBERT']['exercised_stock_options']= 'NaN'
data_dict['BELFER ROBERT']['restricted_stock']= 44093
data_dict['BELFER ROBERT']['restricted_stock_deferred']= -44093
data_dict['BELFER ROBERT']['total_stock_value'] = 'NaN'

data_dict['BHATNAGAR SANJAY']['other']= 'NaN' 
data_dict['BHATNAGAR SANJAY']['expenses']= 137864
data_dict['BHATNAGAR SANJAY']['director_fees']= 'NaN'
data_dict['BHATNAGAR SANJAY']['total_payments']= 137864
data_dict['BHATNAGAR SANJAY']['exercised_stock_options']= 15456290 
data_dict['BHATNAGAR SANJAY']['restricted_stock']= 2604490
data_dict['BHATNAGAR SANJAY']['restricted_stock_deferred']= -2604490
data_dict['BHATNAGAR SANJAY']['total_stock_value']= 15456290


WHALEY DAVID A 18
WROBEL BRUCE 18
LOCKHART EUGENE E 20
THE TRAVEL AGENCY IN THE PARK 18
GRAMM WENDY L 18


/Users/wendy/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [132]:
### Task 3: Create new feature(s)

## create dataframe of dict with features as cols and names as index
#https://stackoverflow.com/questions/13575090/construct-pandas-dataframe-from-items-in-nested-dictionary
from __future__ import division
import numpy as np
import pandas as pd

df_data = pd.DataFrame.from_dict({(i): data_dict[i]
                                  for i in data_dict.keys()},orient='index')

        
##List of Features that are numbers
numbers = list(df_data)  
numbers.remove('email_address') 
numbers.remove('poi')

##Ensure pd.dataframe values are 'float' for mathematical operations
for trait in [numbers]:
    df_data[trait] = df_data[trait].astype('float')

## New features:
 
df_data['take_home'] = df_data['salary'] + df_data['bonus']
df_data['percent_exercised'] = df_data['exercised_stock_options']/df_data['total_stock_value']
df_data['response_rate'] = df_data['from_messages']/df_data['to_messages']
df_data['poi_response'] = df_data['from_this_person_to_poi']/df_data['from_poi_to_this_person']

new_features=['take_home', 'percent_exercised', 'response_rate', 'poi_response']
numbers = numbers + new_features 


##Data Prepocessing: Imputation (replace NaNs)
##Replace inf with NaN
#df_data.dropna(0,'all') -> no rows with all empty features
df_data = df_data.replace(np.inf, np.nan)

imputed_mean = df_data.copy()   
imputed_median = df_data.copy()
imputed_zero = df_data.copy()
for col in numbers:
    ave =  imputed_mean[col].mean()
    imputed_mean[col] = imputed_mean[col].replace(np.nan, ave)
    imputed_median[col] = imputed_median[col].replace(np.nan, ave)
    imputed_zero[col] = imputed_zero[col].replace(np.nan, 0)

#print imputed_mean.describe()
#print imputed_mean.plot.box(rot=90)
#print imputed_zero.describe()
#print imputed_zero.plot.box(rot=90)

## Removed features: 
## 'restricted_stock_deferred', 17 entries; 'loan_advances', 3 entries; 'director_fees',16 entries
##  'deferral_payments, 38 entries (73% NaNs)
##      -limited entries <10%, 2 have no POIs to classify on

my_features = ['poi','salary', 'bonus','total_payments',  
               'exercised_stock_options','restricted_stock','total_stock_value',
               'expenses','deferred_income',
               'long_term_incentive', 'other',
               'to_messages', 'shared_receipt_with_poi','from_messages',      
               'from_this_person_to_poi', 'from_poi_to_this_person',
              # 'restricted_stock_deferred','loan_advances','director_fees', #Removed - too few points
              # 'deferral_payments',
              'take_home', 'percent_exercised', 'response_rate', 'poi_response' # New features
              ]

## Store new features and corrections to my_dataset dictionary for easy export below.
my_data = df_data.to_dict(orient='index')
my_data_median = imputed_median.to_dict(orient='index')
my_data_mean = imputed_mean.to_dict(orient='index')
my_data_zero = imputed_zero.to_dict(orient='index')
my_dataset = imputed_zero.to_dict(orient='index')


In [155]:

print features[0]

[  1.72954100e+06   1.72954100e+06   4.17500000e+06   2.01955000e+05
   3.04805000e+05  -3.08105500e+06   1.52000000e+02   4.70000000e+01
   6.50000000e+01   1.38297872e+00]


In [4]:

### Extract features and labels from dataset for local testing
def feature_extraction(mydata_dict, features_lineup):
    data = featureFormat(mydata_dict, features_lineup, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    return labels,features

### Split sets for cross validation train/test data 
def split_sets(features, labels, test_amt, r_state): 
    from sklearn.cross_validation import train_test_split
    f_train, f_test, l_train, l_test = \
    train_test_split(features, labels, test_size=test_amt, random_state=r_state)
    return f_train, f_test, l_train, l_test   



#labels, features = feature_extraction(my_dataset, my_features)
#features_train, features_test, labels_train, labels_test = split_sets(features, labels, 0.3, 42)

In [150]:
#### from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import SelectPercentile, f_classif, chi2
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.preprocessing import Imputer


labels, features_raw = feature_extraction(my_data, my_features)

imp_mean = Imputer(missing_values=np.nan, strategy='mean')
features= imp_mean.fit_transform(features_raw)
scaler = StandardScaler()
features = scaler.fit_transform(features)
print features[1]

## Rank Features with various Feature Selection Methods
######################
## VarianceThreshold object to rank feature variances
thresholder = VarianceThreshold()
high_variance = thresholder.fit(features)
## List Features with Ranked variances (ascending)
t_vars = thresholder.variances_
t_vars_sort = np.argsort(thresholder.variances_)
print "VarianceThreshold:"
for i in t_vars_sort:
    print i, my_features[i+1], t_vars[i]

######################
## KBest takes Featues, target(labels), select k features
kbest = SelectKBest(f_regression).fit(features, labels)
k_scoresort = np.argsort(kbest.scores_)
k_pvals  = kbest.pvalues_
print "########  SelectKBest:"
for f in k_scoresort:
    print my_features[f+1], 'score: ', kbest.scores_[f], k_pvals[f]

#####################    
## Select Percentile, default selection function: the 10% most significant features
selector = SelectPercentile(f_classif, percentile=10)
selector.fit(features, labels)
scores = -np.log10(selector.pvalues_)
scores /= scores.max()
print "##########  SelectPercentile:"
for f in np.argsort(scores):
    print my_features[f+1], 'score: ', scores[f], k_pvals[f]


[  4.07485195e-16   1.29552474e-15  -2.67230484e-01  -6.07480676e-01
  -4.81157870e-16  -5.22493919e-01  -1.34073716e+00   2.14970267e-16
  -2.01659617e-15   2.10091579e-16   6.85134538e-16  -1.25142943e-15
   6.40762271e-16  -9.20940043e-16  -4.23831393e-16   1.61667248e-15
   1.30977200e+00  -3.79240754e-16  -5.06850054e-16]
VarianceThreshold:
9 other 1.0
2 total_payments 1.0
6 expenses 1.0
15 take_home 1.0
3 exercised_stock_options 1.0
10 to_messages 1.0
17 response_rate 1.0
13 from_this_person_to_poi 1.0
11 shared_receipt_with_poi 1.0
14 from_poi_to_this_person 1.0
18 poi_response 1.0
4 restricted_stock 1.0
1 bonus 1.0
8 long_term_incentive 1.0
5 total_stock_value 1.0
16 percent_exercised 1.0
0 salary 1.0
7 deferred_income 1.0
12 from_messages 1.0
########  SelectKBest:
percent_exercised score:  0.00130552398188 0.971228181585
to_messages score:  0.366007900033 0.54616080033
expenses score:  0.400610688055 0.527799224419
from_messages score:  0.582370663056 0.446659512623
poi_respo

In [6]:
## Review Features: correlation matrix pandas, boxplot, statistics
## https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas

#print imputed_zero.describe()

s= imputed_zero.corr()
s_order = s.unstack().sort_values(ascending=False)

for key, value in s_order.iteritems():
    if value > 0.8 and value !=1:
        print key, value
        
#imputed_zero[my_features].plot.box(rot=90)

('take_home', 'bonus') 0.993993530943
('bonus', 'take_home') 0.993993530943
('total_payments', 'loan_advances') 0.972798552984
('loan_advances', 'total_payments') 0.972798552984
('total_stock_value', 'exercised_stock_options') 0.964537075968
('exercised_stock_options', 'total_stock_value') 0.964537075968
('response_rate', 'from_messages') 0.927433275593
('from_messages', 'response_rate') 0.927433275593
('shared_receipt_with_poi', 'to_messages') 0.881262542007
('to_messages', 'shared_receipt_with_poi') 0.881262542007
('total_payments', 'other') 0.83829180367
('other', 'total_payments') 0.83829180367


In [109]:
### USE top scores from Variance Threshold, SelectKBest/SelectPercentile for order, remove high correlation features
## feature_list = ['poi','total_stock_value','take_home',
                #'exercised_stock_options','bonus',
                #'salary', 'deferred_income','long_term_incentive',
                #'total_payments','restricted_stock','shared_receipt_with_poi',
                # 'expenses','from_poi_to_this_person', 'other',
                # 'poi_response, 'from_this_person_to_poi',
                # 'to_messages', 'from_messages', 'response_rate', 'percent_exercised'
#                ]

## Ranked for Zero imputation
my_features = ['poi', #'total_stock_value', 'take_home',
               'exercised_stock_options','bonus',
               'salary', 'deferred_income','long_term_incentive',
               #'total_payments',
               'restricted_stock', 'shared_receipt_with_poi',
               'expenses', 'from_poi_to_this_person', 'other', 
               #'poi_response', 'from_this_person_to_poi',
               #
              ]

labels, features = feature_extraction(my_dataset, my_features)
#labels, features = feature_extraction(my_data_imputed, my_features)
print "Features: ", len(my_features)-1

Features:  10


In [152]:
### Ranked for Mean/Median
my_features = ['poi', 'exercised_stock_options', 'total_stock_value', 
               'bonus', #'take_home', 
               'salary', #'total_payments',
               'long_term_incentive',#'restricted_stock',
               'deferred_income','other',#'shared_receipt_with_poi',
               'from_poi_to_this_person','from_this_person_to_poi',  
               #'response_rate' ,
               'poi_response', 
               #'expenses','from_messages', 'to_messages', 'percent_exercised'
              ]
labels, features = feature_extraction(my_dataset, my_features)
#labels, features = feature_extraction(my_data_imputed, my_features)
print "Features: ", len(my_features)-1

Features:  10


In [164]:
###########################
## Pipeline for feature reduction with PCA()/SelectKBest
## http://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html
#sphx-glr-auto-examples-compose-plot-compare-reduction-py
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.preprocessing import MinMaxScaler, StandardScaler


labels, features = feature_extraction(my_data_median, my_features)
sss = StratifiedShuffleSplit(labels, 50, test_size = 0.3, random_state = 42)


## Combine PCA and Univariate Selection:
pca = PCA()
selectk = SelectKBest()
united_features = FeatureUnion([("pca", pca),("select", selectk)])

                  
pipe = Pipeline([#("impute",Imputer(strategy = 'mean')),
                 ("scale", StandardScaler()),
                 ("features", united_features)
#                 ('reduce_dim', SelectKBest()),
#                 ('classify', GaussianNB())
])

K_FEATURES_OPTIONS = [1, 5, 9, 10]
S_FUNC = [f_regression, f_classif]
C_OPTIONS = [25]
                  
param_grid = [{'features__pca__n_components': [1,3,5,7,10],
               'features__pca__whiten': [True, False],
               'features__select__k': K_FEATURES_OPTIONS,
               'features__select__score_func': S_FUNC,
               #'reduce_dim': [PCA()],
               #'reduce_dim__n_components': N_FEATURES_OPTIONS,
               'classify__C': C_OPTIONS
              },
              
]
                  
#  {'reduce_dim': [SelectKBest()],
#        'reduce_dim__k': N_FEATURES_OPTIONS,
#        'reduce_dim__score_func': S_FUNC,
#        'classify__C': C_OPTIONS
#   },
                  

grid = GridSearchCV(pipe, param_grid=param_grid, cv=sss)
grid.fit(features, labels)

mean_scores = np.array(grid.cv_results_['mean_test_score'])
print "Pipeline Feature Reductions PCA()/SelectKBest with SCV classifier"
print mean_scores
print grid.best_score_, grid.best_params_
grid.cv_results_['params'][grid.best_index_]

clf1 = grid.best_estimator_

features_train, features_test, labels_train, labels_test = split_sets(features, labels, 0.3, 42)
clf1.fit(features_train,labels_train)

print("Tester Scores:")
test_classifier(clf1, my_data_median, my_features, 1000)


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('features', FeatureUnion(n_jobs=1,
       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('select', SelectKBest(k=10, score_func=<function f_classif at 0x10a386668>))],
       transformer_weights=None))]) does not.

In [ ]:
print my_features
print labels
print features[2]

In [131]:
## Pipeline for classifier selection with one feature reduction
from sklearn import tree, neighbors
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

labels, features = feature_extraction(my_data_median, my_features)
sss = StratifiedShuffleSplit(labels, 50, test_size = 0.3, random_state = 42)
## Combine PCA and Univariate Selection:

pca = PCA(n_components = 1, whiten=False)
selectk = SelectKBest(k=1, score_func = f_regression)
united_features = FeatureUnion([("select", selectk),("pca", pca)])

pipe2 = Pipeline([("scale", MinMaxScaler()),
                 ("features", united_features),
                 ('classify', SVC())
])

#pipe2 = Pipeline([("scale", MinMaxScaler(feature_range=(0, 1))),
#                  ('reduce_dim', PCA(n_components=3)),
#                  ('reduce_dim', SelectKBest(k=1)),
#                  ('classify', GaussianNB())
#])

#N_FEATURES_OPTIONS = [1,2,4]
C_OPTIONS = [1, 5, 25]             #SVC
SPLITS = [2, 10, 20]               #Decision Tree
WEIGHTS = ['distance', 'uniform']  #K Nearest Neighbors
NACHBARN = [5, 15]                 #K Nearest Neighbors
ESTIMATES = [25, 50, 75]           #Ada Boost

param_grid = [
    {
        'classify': [SVC()],
        'classify__C': C_OPTIONS 
    },
    {
        'classify': [GaussianNB()]
    },
    {
        'classify': [tree.DecisionTreeClassifier()],
        'classify__min_samples_split': SPLITS 
    },
    {
        'classify': [neighbors.KNeighborsClassifier()],
        'classify__n_neighbors': NACHBARN,
        'classify__weights': WEIGHTS
    },
    {
        'classify': [RandomForestClassifier()],
        'classify__n_estimators': ESTIMATES
    },
    {
        'classify': [AdaBoostClassifier()],
        'classify__n_estimators': ESTIMATES
    }
]

grid = GridSearchCV(pipe2, param_grid=param_grid, cv= sss)
grid.fit(features, labels)

mean_scores = np.array(grid.cv_results_['mean_test_score'])

print "Pipeline with various classifiers"
print mean_scores
print grid.best_score_, grid.best_params_
print grid.cv_results_['params'][grid.best_index_]

clf2 = grid.best_estimator_

features_train, features_test, labels_train, labels_test = split_sets(features, labels, 0.3, 42)
clf2.fit(features_train,labels_train)

print("Tester Scores:")
test_classifier(clf2, my_data_median, my_features, 1000)

Pipeline with various classifiers
[ 0.88418605  0.88837209  0.8944186   0.86837209  0.81209302  0.83069767
  0.8455814   0.85116279  0.87162791  0.87348837  0.88372093  0.84790698
  0.84744186  0.84744186  0.84976744  0.83813953  0.83162791]
0.894418604651 {'classify__C': 25, 'classify': SVC(C=25, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)}
{'classify__C': 25, 'classify': SVC(C=25, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)}
Tester Scores:
Pipeline(steps=[('scale', MinMaxScaler(copy=True, feature_range=(0, 1))), ('features', FeatureUnion(n_jobs=1,
       transformer_list=[('select', SelectKBest(k=1, score_func=<function f_regression at 0x10a3867d0>)), 

In [ ]:
#########################################################
### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

##http://scikit-learn.org/0.16/auto_examples/model_selection/grid_search_digits.html#example-model-selection-grid-search-digits-py
## 

from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import recall_score, accuracy_score, precision_score

## Load/Prepare dataset
labels, features = feature_extraction(my_data_median, my_features)
sss = StratifiedShuffleSplit(labels, 50, test_size = 0.3, random_state = 42)


pipe3 = Pipeline([#('scale', MinMaxScaler(feature_range=(0, 1))),
#                  ('reduce_dim', PCA(n_components=3)),
                  ('reduce_dim', SelectKBest()),
                  ('classify', SVC())
])

## Set tuning parameters for cross-validation
cees = [5, 7, 10, 15] # C values
kay = [2,4,6,7]

tune_KNNparameters = [{'classify__n_neighbors': [25, 50, 75, 100],
                       'classify__weights':['distance', 'uniform'],
                       'classify__algorithm': ['auto'],
                       'classify__leaf_size':[1,3,5],
                       'classify__p':[1]
                      }
                     ]
tune_SVCparameters = [{'reduce_dim__k': kay,
                       'reduce_dim__score_func':[f_classif, f_regression],
                       'classify__kernel': ['rbf'],
                       'classify__C': cees,
                       'classify__gamma': [1e-5, 1e-6, 'auto']
                      },
                      {'reduce_dim__k': kay,
                       'reduce_dim__score_func':[f_classif, f_regression],
                       'classify__kernel': ['linear'],
                       'classify__C': cees},
                      #{'classify__kernel': ['poly'],   
                      # 'classify__C': cees, 
                      # 'classify__degree':[3, 4, 5],
                      # 'classify__gamma': [1e-5, 1e-6, 'auto']
                      #}
                     ]

tune_RFparameters = [{'n_estimators': [9,10,12,15],
                      'criterion': ['gini','entropy'],
                      'min_samples_split': [2,3,4,5],
#                     'min_samples_leaf': [1,2,3],
                      'max_features': ['auto', 1, 0.5]
                     }
                  ]

tune_ADAparameters = [{#'reduce_dim__k': kay,
                       'classify__n_estimators': [2,5,50,100],
                       'classify__algorithm': ['SAMME','SAMME.R'],
                       'classify__learning_rate': [1, 0.5, 0.25],
                     }
                  ]

print("Parameter Tuning\n")

gs = GridSearchCV(pipe3, tune_SVCparameters, cv=sss)
gs.fit(features, labels)
clf = gs.best_estimator_

features_train, features_test, labels_train, labels_test = split_sets(features, labels, 0.3, 42)
clf.fit(features_train,labels_train)

print("Best parameters:")
print(gs.best_score_, gs.best_params_)
print("\nGrid scores:")
for params, mean_score, scores in gs.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std() * 2, params))
print()


print("Tester Scores:")
test_classifier(clf, my_data_zero, my_features, 1000)




In [ ]:
             
print("Prediction with SKfold shuffle cross validations.")
skfold = KFold(n_splits=5, shuffle=True)
for train_indices, test_indices in skfold.split(labels):
    features_train = [features[i] for i in train_indices]
    features_test = [features[i] for i in test_indices]
    labels_train = [labels[i] for i in train_indices]
    labels_test = [labels[i] for i in test_indices]

    clf_prediction = clf.predict(features_test)
    print "accuracy score: ", \
        accuracy_score(labels_test, clf_prediction)
    print "precision_score: ", \
        precision_score(labels_test, clf_prediction)
    print "recall_score", \
        recall_score(labels_test, clf_prediction)

print("Prediction with Single Split 50/50 validation")
features_train, features_test, labels_train, labels_test = split_sets(features, labels, 0.3, 9)
y_true, y_pred = labels_train, clf.predict(features_train)
print(classification_report(y_true, y_pred))
print()

### Precision Recall and F1 Metrics
def prec_recall(model, pred, labels_test):
    from sklearn.metrics import precision_score, recall_score, f1_score
    y_true = labels_test
    y_pred = pred
    print model 
    print "Precision:", precision_score(y_true, y_pred)  
    print "Recall:", recall_score(y_true, y_pred)  
    print "F1 Score:", f1_score(y_true, y_pred)
    
    
prec_recall(clf, y_pred, y_true)

In [18]:
from sklearn.cross_validation import StratifiedShuffleSplit

PERF_FORMAT_STRING = "\
\tAccuracy: {:>0.{display_precision}f}\tPrecision: {:>0.{display_precision}f}\t\
Recall: {:>0.{display_precision}f}\tF1: {:>0.{display_precision}f}\tF2: {:>0.{display_precision}f}"
RESULTS_FORMAT_STRING = "\tTotal predictions: {:4d}\tTrue positives: {:4d}\tFalse positives: {:4d}\
\tFalse negatives: {:4d}\tTrue negatives: {:4d}"

def test_classifier(clf, dataset, feature_list, folds):
    data = featureFormat(dataset, feature_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    cv = StratifiedShuffleSplit(labels, folds, test_size = 0.3, random_state = 42)
    true_negatives = 0
    false_negatives = 0
    true_positives = 0
    false_positives = 0
    for train_idx, test_idx in cv: 
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []
        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )       
        ### fit the classifier using training set, and test on test set
        clf.fit(features_train, labels_train)
        predictions = clf.predict(features_test)
        for prediction, truth in zip(predictions, labels_test):
            if prediction == 0 and truth == 0:
                true_negatives += 1
            elif prediction == 0 and truth == 1:
                false_negatives += 1
            elif prediction == 1 and truth == 0:
                false_positives += 1
            elif prediction == 1 and truth == 1:
                true_positives += 1
            else:
                print "Warning: Found a predicted label not == 0 or 1."
                print "All predictions should take value 0 or 1."
                print "Evaluating performance for processed predictions:"
                break
    try:
        total_predictions = true_negatives + false_negatives + false_positives + true_positives
        accuracy = 1.0*(true_positives + true_negatives)/total_predictions
        precision = 1.0*true_positives/(true_positives+false_positives)
        recall = 1.0*true_positives/(true_positives+false_negatives)
        f1 = 2.0 * true_positives/(2*true_positives + false_positives+false_negatives)
        f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)
        print clf
        print PERF_FORMAT_STRING.format(accuracy, precision, recall, f1, f2, display_precision = 5)
        print RESULTS_FORMAT_STRING.format(total_predictions, true_positives, false_positives, false_negatives, true_negatives)
        print ""
    except:
        print "Got a divide by zero when trying out:", clf
        print "Precision or recall may be undefined due to a lack of true positive predictions."
        print (total_predictions, true_positives, false_positives, false_negatives, true_negatives)
        print ""

In [ ]:
## Make Classifier 

clf2 = Pipeline([("scale", MinMaxScaler(feature_range=(0, 1))),
                 ('reduce_dim', PCA(n_components=3)),
                 ('classify', neighbors.KNeighborsClassifier(p= 1, n_neighbors=35, 
                                                              algorithm= 'auto', weights='distance',
                                                              leaf_size=3))
                ])
test_classifier(clf2, my_data_zero, my_features, 1000)

In [ ]:
clf = Pipeline([("scale", MinMaxScaler(feature_range=(0, 1))),
                 #('reduce_dim', PCA(n_components=1)),
                 ('classify', neighbors.KNeighborsClassifier(p= 1, n_neighbors=40, 
                                                              algorithm= 'auto', weights='distance',
                                                              leaf_size=1))])
test_classifier(clf, my_data_median, my_features, 1000)

In [ ]:
my_features

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf, my_dataset, features_list)